In [1]:
import pandas as pd
import numpy as np

redWine = pd.read_csv("winequality-red.csv", delimiter=(';'))
whiteWine = pd.read_csv("winequality-white.csv", delimiter=(';'))

In [2]:
redWine['alcohol'] = np.where(redWine['alcohol'].str.count('\.').gt(1), 10.5, redWine['alcohol'])
whiteWine['alcohol'] = np.where(whiteWine['alcohol'].str.count('\.').gt(1), 10.38, whiteWine['alcohol'])

In [3]:
redWine.alcohol = redWine.alcohol.astype(dtype=np.float64)
whiteWine.alcohol = whiteWine.alcohol.astype(dtype=np.float64)

In [4]:
redWine["type"] = 1
whiteWine["type"] = 0
wineDF = redWine.append(whiteWine, ignore_index=True)

In [6]:
wineDF.columns = wineDF.columns.str.replace(' ', '_')

In [7]:
from keras import layers, optimizers, regularizers
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.models import Sequential
from keras.utils import plot_model

import keras.backend as K

from sklearn import preprocessing, model_selection 

Using TensorFlow backend.


In [8]:
wineDF["quality"] =wineDF["quality"].astype(int)
wineDF = pd.get_dummies(wineDF, columns=["quality"])
wineDF.head(5)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,type,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,1,0,0,1,0,0,0,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,1,0,0,1,0,0,0,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,1,0,0,1,0,0,0,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1,0,0,0,1,0,0,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,1,0,0,1,0,0,0,0


In [46]:
X = wineDF.iloc[:,0:12].values # features
Y = wineDF.iloc[:,13:].values # labels
X = preprocessing.normalize(X, axis = 0)

X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X,Y,test_size=0.18)

print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

(5327, 12) (5327, 6) (1170, 12) (1170, 6)


In [47]:
winemod1 = Sequential()

winemod1.add(Dense(30, input_dim=12, activation='relu', name='input',kernel_regularizer=regularizers.l2(0.01)))
winemod1.add(BatchNormalization(momentum=0.99, epsilon=0.001))

winemod1.add(Dense(50, name='hidden1',bias_initializer='zeros'))
winemod1.add(BatchNormalization(momentum=0.99, epsilon=0.001))
winemod1.add(Activation('tanh'))
winemod1.add(Dropout(0.5))

winemod1.add(Dense(100, name='hidden2',bias_initializer='zeros'))
winemod1.add(BatchNormalization(momentum=0.99, epsilon=0.001))
winemod1.add(Activation('tanh'))
winemod1.add(Dropout(0.5))

winemod1.add(Dense(6, name='output',bias_initializer='zeros'))
winemod1.add(BatchNormalization(momentum=0.99, epsilon=0.001))
winemod1.add(Activation('softmax'))

In [48]:
winemod1.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 30)                390       
_________________________________________________________________
batch_normalization_21 (Batc (None, 30)                120       
_________________________________________________________________
hidden1 (Dense)              (None, 50)                1550      
_________________________________________________________________
batch_normalization_22 (Batc (None, 50)                200       
_________________________________________________________________
activation_16 (Activation)   (None, 50)                0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 50)                0         
_________________________________________________________________
hidden2 (Dense)              (None, 100)              

In [49]:
SGD = optimizers.SGD(lr=0.01, nesterov=True)
winemod1.compile(optimizer = SGD, loss = "mean_squared_error", metrics = ["accuracy"])

In [50]:
winemod1.fit(x = X_train, y = Y_train, epochs = 200, verbose=1, batch_size = 64, validation_data=(X_test, Y_test))

Train on 5327 samples, validate on 1170 samples
Epoch 1/200
5327/5327 [==============================] - 1s 224us/step - loss: 0.3173 - accuracy: 0.1778 - val_loss: 0.2950 - val_accuracy: 0.0325
Epoch 2/200
5327/5327 [==============================] - 0s 44us/step - loss: 0.3121 - accuracy: 0.1703 - val_loss: 0.2905 - val_accuracy: 0.0316
Epoch 3/200
5327/5327 [==============================] - 0s 41us/step - loss: 0.3044 - accuracy: 0.1798 - val_loss: 0.2851 - val_accuracy: 0.0419
Epoch 4/200
5327/5327 [==============================] - 0s 43us/step - loss: 0.2987 - accuracy: 0.1932 - val_loss: 0.2794 - val_accuracy: 0.2726
Epoch 5/200
5327/5327 [==============================] - 0s 43us/step - loss: 0.2936 - accuracy: 0.1917 - val_loss: 0.2737 - val_accuracy: 0.3162
Epoch 6/200
5327/5327 [

In [51]:
preds = winemod1.evaluate(x = X_test, y = Y_test)
print()
print ("Loss = " + str(preds[0]))
print ("Accuracy = " + str(preds[1]))

1170/1170 [==============================] - 0s 26us/step

Loss = 0.09933197525067207
Accuracy = 0.5538461804389954
